# Emerging Pest and Pathogens (EPP)

Peter Søgaard Jørgensen [](https://orcid.org/0000-0002-2621-378X) (Sustainability Learning and Research Centre, Kungliga Vetenskapsuniversitetet, Stockholm Resilience Centre)  
Diana Veronica Luna Gonzalez [](https://orcid.org/0000-0003-0194-4137) (Stockholm Resilience Centre)  
Luong Nguyen Thanh [](https://orcid.org/0000-0002-9270-9345) (Sustainability Learning and Research Centre, Kungliga Vetenskapsuniversitetet, Uppsala Antibiotic Centre)  
Melissa Barton [](https://orcid.org/0000-0003-0661-134X) (Stockholm Resilience Centre)  
Kathryn Louise Bjorklund (Stockholm Resilience Centre)  
Ege Pehlivanoglu (Kungliga Vetenskapsuniversitetet)  
July 16, 2024

In [ ]:
import openai
import time
import os
import pickle
import pandas as pd
import json
from sklearn.metrics import (
    confusion_matrix,
    f1_score,
    cohen_kappa_score,
    classification_report,
)
import numpy as np
from itertools import combinations
from dotenv import load_dotenv

In [ ]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
who_data = pd.read_csv("../data/corpus.csv")
sub_data = who_data[0:9]
sub_data

In [17]:
class OpenAIAPI:
    def __init__(self, settings):
        self.settings = settings

    def gpt_response(self, message):
        try:
            response = openai.Completion.create(
                prompt=message,
                temperature=self.settings["temperature"],
                max_tokens=self.settings["max_tokens"],
                model=self.settings["model"],
            )
            gpt_answer = response["choices"][0]["text"].strip("\n").lower()
            return gpt_answer

        except openai.error.RateLimitError as e:
            # Handle rate limit error
            print(f"Rate limited. Error message: {e}")
            # Wait for 60 seconds before retrying
            print("Waiting for 60 seconds to start again...")
            time.sleep(60)
            # Retry the API call
            return self.gpt_response(message)

    def chatgpt_response(self, message):
        try:
            response = openai.ChatCompletion.create(
                model=self.settings["model"],
                messages=[{"role": "user", "content": message}],
            )
            chatgpt_answer = (
                response["choices"][0]["message"]["content"].strip("\n").lower()
            )
            return chatgpt_answer
        except openai.error.RateLimitError as e:
            # Handle rate limit error
            print(f"Rate limited. Error message: {e}")
            # Wait for the recommended duration before retrying
            print("Waiting for 60 seconds to start again...")
            time.sleep(60)
            # Retry the API call
            return self.chatgpt_response(message)

    def generate_response(self, data, prompt, model, ckpt):
        i = 0
        result = []
        if ckpt:
            with open(ckpt, "rb") as f:
                result = pickle.load(f)
            i = len(result)
            data = data[i:]

        for sent in data:
            if len(sent) == 0:
                break

            print(i)
            message = prompt.format(sent)

            if model == "gpt3":
                answer = self.gpt_response(message)
            elif model in ["chatgpt", "gpt4", "gpt-3.5-turbo"]:
                answer = self.chatgpt_response(message)
            else:
                raise ValueError(f"Model {model} not recognized")
            result.append(answer)
            i += 1

            # Save partial response
            CKPT = "./results/ckpt/"
            if not os.path.exists(CKPT):
                print(f"{CKPT} does not exist. Creating...")
                os.mkdir(CKPT)
            with open(f"{CKPT}{model}_result_part.pkl", "wb") as f:
                pickle.dump(result, f)
        print("---Done")
        return result


def load_prompt(dir):
    with open(dir) as f:
        prompt = "".join(f.readlines())
    return prompt


def get_confusion_matrix(data, label1, label2, output_dir):
    true, pred = data[label1], data[label2]
    true = true.to_numpy().reshape(-1)
    pred = pred.to_numpy().reshape(-1)
    mt = confusion_matrix(true, pred)

    with open(output_dir.format("confusion_matrix"), "a") as f:
        f.write(f"---{label1} & {label2} Matrix---\n")
        f.write(np.array2string(mt) + "\n\n")


def get_macrof1_score(data, label1, label2, output_dir):
    true, pred = data[label1], data[label2]
    score = f1_score(true, pred, average="macro")
    report = classification_report(true, pred, digits=3)
    print(f"Macro f1-score between {label1} & {label2}: {round(score, 3)}")
    print("---Classification Report created")

    with open(output_dir.format("macro_f1score"), "a") as f:
        f.write(f"Macro f1-score between {label1} & {label2}: {round(score, 3)}\n")
        f.write(report + "\n")


def get_kappa_score(data, label1, label2, output_dir):
    true, pred = data[label1], data[label2]
    score = cohen_kappa_score(true, pred)
    print(f"Cohen Kappa score between {label1} & {label2}: {round(score, 3)}")

    with open(output_dir.format("cohenKappa"), "a") as f:
        f.write(f"Cohen Kappa score between {label1} & {label2}: {round(score, 3)}\n")


def main(work, model, data, column_name, prompt_text, output_path, checkpoint=None):
    if not os.path.exists(output_path):
        print(f"{output_path} does not exist. Creating...")
        os.mkdir(output_path)

    settings = {
        "temperature": 0.7,
        "max_tokens": 150,
    }

    if model == "gpt3":
        settings["model"] = "text-davinci-003"
    elif model == "gpt4":
        settings["model"] = "gpt-4"
    elif model in ["chatgpt", "gpt-3.5-turbo"]:
        settings["model"] = "gpt-3.5-turbo"
    else:
        raise ValueError(f"Model {model} not recognized")

    # Print column names to help identify the correct column
    print("Available columns:", data.columns)

    # Check if the specified column exists in the dataset
    if column_name not in data.columns:
        raise KeyError(
            f"Column '{column_name}' does not exist in the dataset. Please check the column names."
        )

    if work == "api":
        data = data[column_name]
        prompt = prompt_text
        output_file_path = os.path.join(output_path, f"{model}-results.csv")

        api = OpenAIAPI(settings)
        result = api.generate_response(data, prompt, model, checkpoint)

        df = pd.DataFrame({column_name: data, f"{model}-results": result})
        df.to_csv(output_file_path, index=False)
        print("Saved result file")

    elif work == "quant":
        colnames = data.columns[1:]
        output_file_path_template = os.path.join(output_path, f"{model}-quant.txt")
        for label1, label2 in combinations(colnames, 2):
            if label1 == "label":
                get_confusion_matrix(data, label1, label2, output_file_path_template)
                get_macrof1_score(data, label1, label2, output_file_path_template)
                get_kappa_score(data, label1, label2, output_file_path_template)
            elif label1 != "label":
                get_kappa_score(data, label1, label2, output_file_path_template)

In [ ]:
# Define parameters
work = "api"  # or 'quant'
model = "gpt-3.5-turbo"  # or 'chatgpt', 'gpt4'
prompt_text = "Extract all causes and effects pairs that are drivers of the emergence and transmission of the disease: {}"
output_path = "./results/"
checkpoint = None  # or specify a checkpoint path
column_name = "Text"  # specify the column to be used for the analysis

# Call the main function
main(work, model, sub_data, column_name, prompt_text, output_path, checkpoint)